In [5]:
import pandas as pd
import setting
import numpy as np

import datetime
from dateutil.relativedelta import relativedelta



#load portfolio weight

p_weight=pd.read_pickle(setting.datapath_pf_weight+'pfweight_FF_L_VOL_24W_lag_lag_False1963-07-2016-12.pkl')


start_date=datetime.datetime(1963,7,1)

end_date=datetime.datetime(2016,12,1)

p_return=pd.read_csv(setting.datapath_prepared+'M_RET.csv')
p_return['date']=pd.to_datetime(p_return['date'])

t_cost=pd.DataFrame(columns=[1,2,3,4,5])

while (start_date<end_date):
    
    next_month=start_date+relativedelta(months=+1)

    weight=p_weight[start_date]

    weight_nextm=p_weight[next_month]

    
    
    month_return=p_return[p_return['date']==start_date]
    temp=pd.DataFrame(index=[start_date],columns=[1,2,3,4,5])
    for i in range(1,6):
        
        m_return=month_return[month_return['PERMNO'].isin(list(weight[i-1].index.values))]
        portfolio=m_return[['PERMNO','M_RET']]
        portfolio=portfolio.set_index('PERMNO')

        portfolio=pd.concat([portfolio,weight[i-1]],axis=1,join='inner')

        portfolio['new weight']=portfolio['M_RET']*portfolio['weight']

        total=portfolio['new weight'].sum()

        portfolio['new weight']=portfolio['new weight']/total

        port_change=pd.concat([weight_nextm[i-1],portfolio['new weight']],axis=1)
        port_change=port_change.fillna(0)
        port_change['rebalance']=np.abs(port_change['new weight']-port_change['weight'])
        weight_change=port_change['rebalance'].sum()
        temp.loc[start_date,i]=weight_change
    t_cost=pd.concat([t_cost,temp],axis=0)
    start_date=next_month
    
t_cost.to_excel(setting.datapath+'transaction cost pfweight_FF_L_VOL_24W_lag_lag_False1963-07-2016-12.xlsx')

In [ ]:
#load portfolio return
p_return=pd.read_csv(setting.datapath_prepared+'M_RET.csv')
p_return['date']=pd.to_datetime(p_return['date'])

In [ ]:

month_return=p_return[p_return['date']==start_date]

m_return=month_return[month_return['PERMNO'].isin(list(weight[0].index.values))]
portfolio=m_return[['PERMNO','M_RET']]
portfolio=portfolio.set_index('PERMNO')

portfolio=pd.concat([portfolio,weight[0]],axis=1,join='inner')

In [ ]:
portfolio['new weight']=portfolio['M_RET']*portfolio['weight']

total=portfolio['new weight'].sum()

portfolio['new weight']=portfolio['new weight']/total


In [ ]:
port_change=pd.concat([weight_nextm[0],portfolio['new weight']],axis=1)
port_change=port_change.fillna(0)
port_change['rebalance']=np.abs(port_change['new weight']-port_change['weight'])
weight_change=port_change['rebalance'].sum()
print (weight_change)

In [ ]:
#Check if we only use differences in beginning portfolio weight

port_weight_change=pd.concat([weight[0],weight_nextm[0]],axis=1)
port_weight_change.columns=['old','new']

port_weight_change=port_weight_change.fillna(0)
port_weight_change['rebalance']=np.abs(port_weight_change['old']-port_weight_change['new'])
weight_change1=port_weight_change['rebalance'].sum()
print (weight_change1)